In [ ]:
#SINTETIZADOR  WENO DE SEA (en fourier)

#Autor: Sebastián Villagra
#Fecha última actualización: no me acuerdo pero fue como en diciembre de 2020

#IMPORTANTE: la funcion ipd.Audio() de IPython.display solo funciona en Google Colab/Jupyter

In [ ]:
#Primero defino la función que genera las frecuencias de cada nota 
#A partir de su semitono y escala

import numpy as np
def gen_frec_nota(nombre_nota,escala):
    #referencia: A_4ta= 440 Hz
    if nombre_nota== "A":
        i= 0
    elif nombre_nota== "A#":
        i= 1
    elif nombre_nota== "B":
        i= 2
    elif nombre_nota== "C":
        i= 3
    elif nombre_nota== "C#":
        i= 4
    elif nombre_nota== "D":
        i= 5
    elif nombre_nota== "D#":
        i= 6
    elif nombre_nota== "E":
        i= 7
    elif nombre_nota== "F":
        i= 8
    elif nombre_nota== "F#":
        i= 9
    elif nombre_nota== "G":
        i= 10
    elif nombre_nota== "G#":
        i= 11

    if nombre_nota == "A" or nombre_nota == "A#" or nombre_nota == "B":
        base = 440/(2**(4-escala))
    else:
        base = 440/(2**(5-escala))

    frec = base*(2**(i/12))
    return frec

In [ ]:
##############################
# Tono puro
##############################

#Ahora la función que entrega la senal en frecuencia
import numpy as np 
import matplotlib.pyplot as plt 
from scipy import signal 
def senal_ft(nota, escala, figura, fs):
    t = tempo*figura
    frecuencia_nota= gen_frec_nota(nota,escala)
    N= int(t*fs)  #OJO el cálculo de dentro es float, debe ser entero
    U= fs/N  

    k = frecuencia_nota//U
    senal_1horquilla = signal.unit_impulse(N,int(k))   #Senal con un impulso en la k-ésima posición
    senal_1h_inversa = senal_1horquilla[::-1]          #Creo otra senal invertida
    senal_2horquillas = senal_1horquilla+senal_1h_inversa #Las sumo para tener ambas horquillas
    senal_2horquillas *= 10000
    return senal_2horquillas

In [ ]:
#######################
# Instrumento 1
#######################

#Para los instrumentos, una función diferente que parte de la misma base
#que el tono puro
#La alternativa era poner condiciones en la funcion original, pero no me parece.
#Las condiciones irán en el ejecutor

def senal_ft_1(nota,escala,figura,fs):
    t = tempo*figura
    frecuencia_nota= gen_frec_nota(nota,escala)
    N= int(t*fs)  #OJO el cálculo de dentro es float, debe ser entero
    U= fs/N  

    #Ahora creo los armónicos
    frec_armonico_1= frecuencia_nota*2
    frec_armonico_2= frecuencia_nota*2.5

    k = frecuencia_nota//U
    k1= frec_armonico_1//U
    k2= frec_armonico_2//U

    impulso_principal = signal.unit_impulse(N,int(k))   #Senal con un impulso en la k-ésima posición
    impulso_arm_1= signal.unit_impulse(N,int(k1))        #Lo mismo pero para el primero armonico
    impulso_arm_2= signal.unit_impulse(N,int(k2))        #Lo mismo pero para el segundo armonico

    impulso_principal *= 20000                          #Amplifico el tono principal
    impulso_arm_1 *= 10000                               #Amplifico el armonico, pero por un valor menor (caracterizar) 
    impulso_arm_2 *= 4500                                #Amplifico el armonico, pero por un valor menor y distinto

    senal_nosim = impulso_principal + impulso_arm_1 + impulso_arm_2         #sumno los tres impulsos 
    senal_inversa = senal_nosim[::-1]                                    #Creo otra senal invertida
    senal_simetrica = senal_nosim+senal_inversa                          #Las sumo para tener una senal simetricay transformable

    return senal_simetrica

In [ ]:
####################
# Instrumento 2
####################

#A este le pondré cuatro armónicos

def senal_ft_2(nota,escala,figura,fs):
    t = tempo*figura
    if escala == 0:
      frecuencia_nota=0
    frecuencia_nota= gen_frec_nota(nota,escala)
    N= int(t*fs)  #OJO el cálculo de dentro es float, debe ser entero
    U= fs/N  

    #Ahora creo los armónicos
    frec_armonico_1= frecuencia_nota*0.5
    frec_armonico_2= frecuencia_nota*1.5
    frec_armonico_3= frecuencia_nota*3
    frec_armonico_4= frecuencia_nota*4

    k = frecuencia_nota//U
    k1= frec_armonico_1//U
    k2= frec_armonico_2//U
    k3= frec_armonico_3//U
    k4= frec_armonico_4//U

    impulso_principal = signal.unit_impulse(N,int(k))   #Senal con un impulso en la k-ésima posición
    impulso_arm_1= signal.unit_impulse(N,int(k1))        #Lo mismo pero para el primero armonico
    impulso_arm_2= signal.unit_impulse(N,int(k2))        #Lo mismo pero para el segundo armonico
    impulso_arm_3= signal.unit_impulse(N,int(k3))
    impulso_arm_4= signal.unit_impulse(N,int(k4))

    impulso_principal *= 20000                          #Amplifico el tono principal
    impulso_arm_1 *= 14000                               #Amplifico el armonico, pero por un valor menor (caracterizar) 
    impulso_arm_2 *= 5000                                #Amplifico el armonico, pero por un valor menor y distinto
    impulso_arm_3 *= 9000
    impulso_arm_4 *= 1000

    senal_nosim = impulso_principal + impulso_arm_1 + impulso_arm_2 + impulso_arm_3 + impulso_arm_4 
    senal_inversa = senal_nosim[::-1]                                    #Creo otra senal invertida
    senal_simetrica = senal_nosim+senal_inversa                          #Las sumo para tener una senal simetricay transformable

    return senal_simetrica

In [ ]:
#Programa entrada de notas
#Para Jupyter

tempo = 0.5
fs= 8192   #mayor a 7903 e idealmente potencia de 2
instrumento = 0   #0, 1 ó 2

#Cada nota debe ser una lista con la Letra (Mayúscula, String),
#escala (natural) y figura (por ahora representada por un racional).

notas_play=[["D",4,1],["G",4,2],["A",4,1.5],["A#",4,0.5],["C",5,0.5],["A#",4,1],["D",4,1],["D",0,1.5],["D",4,0.5],["G",4,1.5],
            ["A",4,0.5],["A#",4,0.5],["D",4,0.5],["A#",4,0.5],["G",4,0.5],["D",5,0.5],["C",5,2],["D",0,1],["D",4,1],["G",4,1.5],
            ["A#",4,1],["G",4,0.5],["D",5,1],["A#",4,0.5],["G",5,2],["G",4,1],["A#",4,0.5],["A",4,0.5],["G",4,0.5],["D",5,1.5],
            ["A",4,0.5],["A#",4,0.5],["G",4,0.5],["D",4,1],["D",4,1],["D",4,0.5],["G",4,4]]  #Mira esa melodía

senales_melodia=[]

for el in notas_play:

  if instrumento ==0:
    s_ft=senal_ft(el[0],el[1],el[2],fs)
    senales_melodia.append(np.fft.ifft(s_ft))

  elif instrumento ==1:
    s_ft=senal_ft_1(el[0],el[1],el[2],fs)
    s_tiempo = np.fft.ifft(s_ft)

    x=np.arange(0,tempo*el[2],1/fs)
    envoltura= abs(np.sin(40*x+1)/(x+1))+0.4
    audio = envoltura*s_tiempo
    audio= np.int16(audio*1000)
    senales_melodia.append(audio)


  elif instrumento ==2: 
    s_ft=senal_ft_2(el[0],el[1],el[2],fs)
    s_tiempo = np.fft.ifft(s_ft)

    x=np.arange(0,tempo*el[2],1/fs)      #creo la lista de variable indep. para la funcion envolvente
    envoltura = np.exp(-6*x-1)
    audio = envoltura*s_tiempo
    audio= np.int16(audio*1000)
    senales_melodia.append(audio)   #le toma la inversa y la agrega a la lista
                                           #ahora tengo una lista con senales en el tiempo
                                           #se pueden reproducir, para el audio completo se concatena
audio_conc=np.concatenate(senales_melodia)  #la entrada es una lista, por lo que
                                            #la función concatena los vectores

In [ ]:
#Reproduce la salida solo en jupyter/collab
import IPython.display as ipd
ipd.Audio(audio_conc, rate=fs)

/usr/local/lib/python3.6/dist-packages/IPython/lib/display.py:123: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(data, dtype=float)


In [ ]:
# Aplicar ruido ambiente a la salida y exportar

from scipy.io import wavfile
import soundfile as sf

#Se convoluciona la señal del audio con la respuesta al impulso (en audio) dentro de algun lugar.
#El audio es un balazo grabado con micrófonos qls de calidad dentro de la sala de interés.
Ambiente = 1

if Ambiente ==0:

elif Ambiente ==1: #Gimnasio
  data,fs = sf.read("sportscentre_cardioid.wav")
  out= np.convolve(audio_conc,data,mode="same")
  wavfile.write("ej_Ambiente_1",fs,out)

elif Ambiente ==2: #Parque
  data,fs=sf.read("koli_summer_site1_1way_mono.wav")
  out= np.convolve(audio_conc,data,mode="same")
  wavfile.write("ej_Ambiente_2",fs,out)